<a href="https://colab.research.google.com/github/cmaijala/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization_h5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import our dependencies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import matplotlib.pyplot as plt

# 1. Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# 2. Drop non-beneficial columns ('EIN', 'NAME', 'STATUS')
application_df = application_df.drop(columns=['EIN', 'NAME', 'STATUS'])

In [5]:
# 3. Binning rare categories for 'APPLICATION_TYPE' and 'CLASSIFICATION'
# For 'APPLICATION_TYPE'
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
rare_types = application_type_counts[application_type_counts < 500].index  # Adjust the threshold if needed
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(rare_types, 'OTHER')

# For 'CLASSIFICATION'
classification_counts = application_df['CLASSIFICATION'].value_counts()
rare_classifications = classification_counts[classification_counts < 1000].index  # Adjust threshold if needed
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(rare_classifications, 'OTHER')


In [6]:
# 4. Apply log transformation to 'ASK_AMT' to compress its range
application_df['ASK_AMT_LOG'] = np.log1p(application_df['ASK_AMT'])  # log(1 + x) to avoid log(0) issues

In [8]:
# 5. Separate features (X) and target (y)
X = application_df.drop(columns=['IS_SUCCESSFUL', 'ASK_AMT'])  # Exclude target and any non-useful columns
y = application_df['IS_SUCCESSFUL']

In [9]:
# 6. One-hot encode categorical columns
X = pd.get_dummies(X)

In [10]:
# 7. Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
# 8. Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
# 9. Build the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_dim=X_train_scaled.shape[1]),  # Input layer + first hidden layer
    tf.keras.layers.Dense(64, activation='relu'),  # Second hidden layer
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer (sigmoid activation for binary classification)
])


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# 10. Train the model
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test))

Epoch 1/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7135 - loss: 0.5799 - val_accuracy: 0.7258 - val_loss: 0.5630
Epoch 2/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7305 - loss: 0.5544 - val_accuracy: 0.7235 - val_loss: 0.5614
Epoch 3/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7352 - loss: 0.5450 - val_accuracy: 0.7255 - val_loss: 0.5612
Epoch 4/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7222 - loss: 0.5543 - val_accuracy: 0.7238 - val_loss: 0.5682
Epoch 5/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7319 - loss: 0.5486 - val_accuracy: 0.7273 - val_loss: 0.5600
Epoch 6/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7359 - loss: 0.5459 - val_accuracy: 0.7264 - val_loss: 0.5581
Epoch 7/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7305 - loss: 0.5462 - val_accuracy: 0.7239 - val_loss: 0.5580
Epoch 8/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7369 - loss: 0.5434 - val_accuracy: 0.

In [15]:
# 11. Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7317 - loss: 0.5665
Test Loss: 0.567645251750946
Test Accuracy: 0.728863000869751


In [16]:
# 12. Save the model
model.save('AlphabetSoupCharity_Optimization.h5')